## Problem Definition
Carryout Topic modelling upon the text to find these hidden topics and their supporting words by maximising the posterior
probability of the whole corpus given the topics and words.

### The steps I am going to take to solve the problem are as follows-
  *  Import packages.
  *  Import document.
  *  Clean data.
  *  Create a dictionary form the clean words.
  *  Using BoW model create document-term matrix
  *  Generate lda model.
  *  Find number of topics = 2
  *  Evaluate the results using graphical representation. 

## Implemmentation.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim import models, corpora


In [ ]:
documents = [
  """
  Artificial intelligence (AI), sometimes called machine
  intelligence, is intelligence demonstrated by machines, unlike
  the natural intelligence displayed by humans and animals. Leading
  AI textbooks define the field as the study of "intelligent
  agents": any device that perceives its environment and takes
  actions that maximize its chance of successfully achieving its
  goals. Colloquially, the term "artificial intelligence" is often
  used to describe machines (or computers) that mimic "cognitive"
  functions that humans associate with the human mind, such
  as "learning" and "problem solving".
  """,
  """
  Association football, more commonly known as football or
  soccer, is a team sport played with a spherical ball between
  two teams of 11 players. It is played by approximately 250
  million players in over 200 countries and dependencies, making it
  the world's most popular sport. The game is played on a
  rectangular field called a pitch with a goal at each end. The
  object of the game is to outscore the opposition by moving the
  ball beyond the goal line into the opposing goal. The team with
  the higher number of goals wins the game.  
  """
]

In [ ]:
# Clean the data by using stemming and stopwords removal
nltk.download('stopwords')
stemmer = SnowballStemmer('english')
stop_words = stopwords.words('english')
texts = [
  [stemmer.stem(word) for word in document.lower().split() if word not in stop_words]
  for document in documents
  ]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Create a dictionary from the words
dictionary = corpora.Dictionary(texts)

# Create a document-term matrix
doc_term_mat = [dictionary.doc2bow(text) for text in texts]

# Generate the LDA model 
num_topics = 2
ldamodel = models.ldamodel.LdaModel(doc_term_mat, 
        num_topics=num_topics, id2word=dictionary, passes=25)


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
num_words = 5
for i in range(num_topics):
  print(ldamodel.print_topic(i, topn=num_words))

print('\nTop ' + str(num_words) + ' contributing words to each topic:')
for item in ldamodel.print_topics(num_topics=num_topics, num_words=num_words):
    print('\nTopic', item[0])
    list_of_strings = item[1].split(' + ')
    for text in list_of_strings:
        details = text.split('*')
        print("%-12s:%0.2f%%" %(details[1], 100*float(details[0])))


0.036*"goal" + 0.036*"play" + 0.036*"team" + 0.026*"game" + 0.026*"ball"
0.035*"intellig" + 0.035*"human" + 0.025*"machin" + 0.015*"field" + 0.015*"call"

Top 5 contributing words to each topic:

Topic 0
"goal"      :3.60%
"play"      :3.60%
"team"      :3.60%
"game"      :2.60%
"ball"      :2.60%

Topic 1
"intellig"  :3.50%
"human"     :3.50%
"machin"    :2.50%
"field"     :1.50%
"call"      :1.50%


In [ ]:
new_docs = [
  """
  Jager thinks this is just the start of AI eating the beautiful
  game. “We have a dedicated team that focuses only on artificial
  intelligence and machine learning for sports teams,” he
  says. “That is not only for soccer, but for Formula One and
  American football. We have a baseball team, and we're talking
  right now with cricket teams.”
  """
]

new_texts = [
  [stemmer.stem(word) for word in document.lower().split() if word not in stop_words]
  for document in new_docs
  ]
new_doc_term_mat = [dictionary.doc2bow(text) for text in new_texts]

vector = ldamodel[new_doc_term_mat]
print(vector[0])


[(0, 0.49326453), (1, 0.50673556)]


### My topic for anaysis from scratch

In [ ]:

import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
Doc = [
          """
          Experts usually rely on common sense when they solve problems. They also use
vague and ambiguous terms. For example, an expert might say, ‘Though the
power transformer is slightly overloaded, I can keep this load for a while.’ Other
experts have no difficulties with understanding and interpreting this statement
because they have the background to hearing problems described like this.
However, a knowledge engineer would have difficulties providing a computer
with the same level of understanding. How can we represent expert knowledge
that uses vague and ambiguous terms in a computer? Can it be done at all?
This chapter attempts to answer these questions by exploring the fuzzy set
theory (or fuzzy logic). We review the philosophical ideas behind fuzzy logic,
study its apparatus and then consider how fuzzy logic is used in fuzzy expert
systems.
""",
"""
Let us begin with a trivial, but still basic and essential, statement: fuzzy logic is
not logic that is fuzzy, but logic that is used to describe fuzziness. Fuzzy logic
is the theory of fuzzy sets, sets that calibrate vagueness. Fuzzy logic is based on
the idea that all things admit of degrees. Temperature, height, speed, distance,
beauty – all come on a sliding scale. The motor is running really hot. Tom is
a very tall guy. Electric cars are not very fast. High-performance drives require
very rapid dynamics and precise regulation. Hobart is quite a short distance
from Melbourne. Sydney is a beautiful city. Such a sliding scale often makes it
impossible to distinguish members of a class from non-members. When does a
hill become a mountain?
"""
]

### Importing my stops words from nltk

In [ ]:
stopwords = stopwords.words("english")

In [ ]:
#  Going to look at the words I just imported
print(stopwords)
# Kool

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Since my data already loaded.  I am going to take a peek at the first 70 charaters
print (Doc[0][0:100])


          Experts usually rely on common sense when they solve problems. They also use
vague and am


In [ ]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in Doc:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(texts)


/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [ ]:
# Now to compare the results withthe above.
print (lemmatized_texts[0][0:90])

expert usually rely common sense when solve problem also use vague ambiguous term example 


In [ ]:
#  Reducing text to individual word and removing stops 
def gen_words(Doc):
    final = []
    for text in Doc:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:10])  #  Lets see if it worked

['expert', 'usually', 'rely', 'common', 'sense', 'when', 'solve', 'problem', 'also', 'use']


In [ ]:
#  Create a BoW and its freqency
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])


[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 5), (19, 1)]


In [ ]:
# Let us now look at the first tuple
word = id2word[[0][:1][0]]
print (word)

all


In [ ]:
# Create a dictionary from the words
dictionary = corpora.Dictionary(texts)

# Create a document-term matrix
doc_term_mat = [dictionary.doc2bow(text) for text in texts]

# Generate the LDA model 
num_topics = 2
ldamodel = models.ldamodel.LdaModel(doc_term_mat, 
        num_topics=num_topics, id2word=dictionary, passes=25)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

### Vector Projection

In [ ]:
num_words = 5
for i in range(num_topics):
  print(lda_model.print_topic(i, topn=num_words))

print('\nTop ' + str(num_words) + ' contributing words to each topic:')
for item in lda_model.print_topics(num_topics=num_topics, num_words=num_words):
    print('\nTopic', item[0])
    list_of_strings = item[1].split(' + ')
    for text in list_of_strings:
        details = text.split('*')
        print("%-12s:%0.2f%%" %(details[1], 100*float(details[0])))

0.053*"fuzzy" + 0.053*"logic" + 0.032*"very" + 0.022*"distance" + 0.022*"member"
0.048*"expert" + 0.048*"fuzzy" + 0.030*"logic" + 0.030*"use" + 0.030*"can"

Top 5 contributing words to each topic:

Topic 4
"logic"     :0.90%
"fuzzy"     :0.90%
"very"      :0.90%
"scale"     :0.90%
"slide"     :0.90%

Topic 1
"expert"    :4.80%
"fuzzy"     :4.80%
"logic"     :3.00%
"use"       :3.00%
"can"       :3.00%


In [ ]:
new_docs = [
  """
  Fuzzy or multi-valued logic was introduced in the 1930s by Jan Lukasiewicz, a
  Polish logician and philosopher (Lukasiewicz, 1930). He studied the mathematical 
  representation of fuzziness based on such terms as tall, old and hot. While
  classical logic operates with only two values, 1 (true) and 0 (false), Lukasiewicz
  introduced logic that extended the range of truth values to all real numbers in
  the interval between 0 and 1. He used a number in this interval to represent the
  possibility that a given statement was true or false. For example, the possibility
  that a man 181 cm tall is really tall might be set to a value of 0.86. It is likely that
  the man is tall. This work led to an inexact reasoning technique often called
  possibility theory.
 
  """
  ]

new_texts = [
  [stemmer.stem(word) for word in document.lower().split() if word not in stopwords]
  for document in new_docs
  ]
new_doc_term_mat = [dictionary.doc2bow(text) for text in new_texts]

vector = ldamodel[new_doc_term_mat]
print(vector[0])



[(0, 0.42149878), (1, 0.5785012)]


###Graphical representation 

In [ ]:
!pip install pyLDAvis==2.1.2
import pyLDAvis.gensim


### Graphical representation of my topic

In [ ]:
# Visualize my topic
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=11)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.008276  0.110625       1        1  52.758030
0      0.023550 -0.107655       2        1  47.167715
2     -0.005438  0.001245       3        1   0.036503
3     -0.005260 -0.001909       4        1   0.019501
4     -0.004576 -0.002306       5        1   0.018251, topic_info=         Term      Freq     Total Category  logprob  loglift
18     expert  4.000000  4.000000  Default  11.0000  11.0000
20      fuzzy  8.000000  8.000000  Default  10.0000  10.0000
30      logic  6.000000  6.000000  Default   9.0000   9.0000
8         can  2.000000  2.000000  Default   8.0000   8.0000
56        use  3.000000  3.000000  Default   7.0000   7.0000
..        ...       ...       ...      ...      ...      ...
45        set  0.000262  2.636172   Topic5  -4.7199  -0.6079
91     member  0.000261  1.851025   Topic5  -4.7248  -0.2592
75   distance  0.000261  1.851057   Topic5  -4.7250  -0.2594
59       when  0.000260  1.868855   Topic5  -4.7265  -0.2705
48  statement  0.000260  1.872461   Topic5  -4.7286  -0.2744

[106 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.910022            all
2         1  1.056984      ambiguous
3         1  0.913332         answer
5         1  0.909683             at
7         1  0.910544     background
64        2  0.923551          basic
67        2  0.923563         become
69        2  0.924729      calibrate
8         1  1.123613            can
70        2  0.923697            car
72        2  0.923571          class
11        1  1.057296       computer
12        1  0.910570       consider
74        2  0.924338         degree
13        1  0.535890       describe
13        2  0.535890       describe
14        1  1.058207     difficulty
75        2  1.080464       distance
76        2  0.924238    distinguish
15        1  0.909757             do
18        1  0.942069         expert
82        2  0.924459      fuzziness
20        1  0.496239          fuzzy
20        2  0.496239          fuzzy
84        2  0.923654         height
85        2  0.924347           high
87        2  0.924225            hot
22        1  1.061590            how
27        1  1.062406      knowledge
30        1  0.460580          logic
30        2  0.614107          logic
90        2  0.924438           make
31        1  0.909772            may
91        2  1.080482         member
92        2  0.923721          motor
97        2  0.924595        precise
36        1  1.061985        problem
100       2  0.924410     regulation
101       2  0.924378        require
43        1  0.909500            say
103       2  1.081405          scale
45        1  0.379338            set
45        2  0.758676            set
105       2  1.081391          slide
48        1  0.534057      statement
48        2  0.534057      statement
107       2  0.924585          still
49        1  0.911391          study
50        1  0.910209         system
108       2  0.923477           tall
51        1  1.061519           term
52        1  0.916797           then
55        1  1.056921  understanding
56        1  0.871373            use
56        2  0.290458            use
58        1  1.059569          vague
113       2  0.764043           very
59        1  0.535087           when
59        2  0.535087           when, R=11, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4, 5])

###Graphical representation of the first analysis

**Conclusion** There are to distinct clusters with no overlapping occuring between them in both cases.  Also, in both cases we see that the overal term frequency words were meaning full.  Hence the topics were pretty well harmnized and unique.  For example in my analysis we had words such as fuzzy, logic, expert, etc which are uniqure to the cluster.  Therefore the vector analsis looks correct.  Words such as go,and, is, etc which has no real value in cluster formation is excluded.  In the first Text analysis we see an overal term frequency of 5 words with the estimated term frequency with the topic being 3.  
